# FGSM

In [ ]:
from advsecurenet.attacks.gradient_based import FGSM
from advsecurenet.shared.types.configs.attack_configs import FgsmAttackConfig
from advsecurenet.shared.types.configs.attack_configs.attacker_config import AttackerConfig
from advsecurenet.attacks.attacker import Attacker
from advsecurenet.shared.types.configs.device_config import DeviceConfig


import numpy as np
import time


import torch
import torchvision
import torchvision.transforms as transforms
from torchvision.models import resnet50


from art.attacks.evasion import FastGradientMethod
from art.estimators.classification import PyTorchClassifier

from cleverhans.torch.attacks.fast_gradient_method import fast_gradient_method

import foolbox as fb

from advertorch.attacks import FGSM as advertorch_FGSM


In [ ]:
# Data transforms
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

# Load CIFAR-10 dataset
trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=128, shuffle=True, num_workers=2)

# Load ResNet-50 model
model = resnet50(pretrained=True).eval()


### IBM ART

In [ ]:
# ART FGSM
classifier = PyTorchClassifier(model=model, clip_values=(0, 1), loss=torch.nn.CrossEntropyLoss(), optimizer=None, input_shape=(3, 32, 32), nb_classes=10)
attack = FastGradientMethod(estimator=classifier, eps=0.3)
start_time = time.time()
for data in trainloader:
    inputs, labels = data
    adv_inputs = attack.generate(x=inputs.numpy())
end_time = time.time()
print("ART FGSM Attack Time:", end_time - start_time)



### CleverHans

In [ ]:
# CleverHans FGSM
start_time = time.time()
for data in trainloader:
    inputs, labels = data
    inputs, labels = inputs.cuda(), labels.cuda()
    adv_inputs = fast_gradient_method(model, inputs, eps=0.3, norm=np.inf)
end_time = time.time()
print("CleverHans FGSM Attack Time:", end_time - start_time)

### AdvSecureNet

In [ ]:
device = DeviceConfig(processor="cuda")

# Define the fgsm config
fgsm_config = FgsmAttackConfig(
    targeted=False,
    epsilon=0.1,
    device=device,
)

# Now we can define the attack
attack = FGSM(config=fgsm_config)  

# define the attacker 
attacker_config = AttackerConfig(
    model=model,
    attack= attack,
    dataloader=trainloader,
    device=device,
    return_adversarial_images=True,
)
attacker = Attacker(config=attacker_config)

start_time = time.time()
adv_imgs = attacker.execute()
end_time = time.time()
print("AdvSecureNet FGSM Attack Time:", end_time - start_time)

### ARES

In [ ]:
start_time = time.time()
!python run_attack.py --gpu 0 --batchsize 20 --num_workers 16  --model_name 'resnet50' --attack_name 'fgsm' --dataset 'cifar10' --data_dir ./data 
end_time = time.time()
print("Ares FGSM Attack Time:", end_time - start_time)

### FoolBox

In [ ]:

attack = fb.attacks.LinfFastGradientAttack()
preprocessing = dict()
bounds = (-1, 1)
fmodel = fb.PyTorchModel(model, bounds=bounds, preprocessing=preprocessing)

start_time = time.time()
for data in trainloader:
    inputs, labels = data
    inputs, labels = inputs.cuda(), labels.cuda()
    adv_inputs = attack(fmodel, inputs, labels, epsilons=[0.3])
    
end_time = time.time()

print("Foolbox FGSM Attack Time:", end_time - start_time)
    

### AdverTorch

In [ ]:
adversary = advertorch_FGSM(model, eps=0.3)

start_time = time.time()
for data in trainloader:
    inputs, labels = data
    inputs, labels = inputs.cuda(), labels.cuda()
    adv_inputs = adversary.perturb(inputs, labels)
end_time = time.time()

print("Advertorch FGSM Attack Time:", end_time - start_time)

# PGD-20

In [ ]:

from advsecurenet.attacks.gradient_based import FGSM, PGD
from advsecurenet.shared.types.configs.attack_configs import FgsmAttackConfig, PgdAttackConfig
from advsecurenet.shared.types.configs.attack_configs.attacker_config import AttackerConfig
from advsecurenet.attacks.attacker import Attacker
from advsecurenet.shared.types.configs.device_config import DeviceConfig


import numpy as np
import time


import torch
import torchvision
import torchvision.transforms as transforms
from torchvision.models import resnet50


from art.attacks.evasion import FastGradientMethod, ProjectedGradientDescent
from art.estimators.classification import PyTorchClassifier

from cleverhans.torch.attacks.fast_gradient_method import fast_gradient_method
from cleverhans.torch.attacks.projected_gradient_descent import projected_gradient_descent

import foolbox as fb

from advertorch.attacks import PGDAttack as AdvertorchPGDAttack

from tqdm.auto import tqdm, trange


In [ ]:
# Data transforms
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

# Load CIFAR-10 dataset
trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=128, shuffle=True, num_workers=2)

# Load ResNet-50 model
model = resnet50(pretrained=True).eval()


### IBM ART

In [ ]:
# ART FGSM
classifier = PyTorchClassifier(model=model, clip_values=(0, 1), loss=torch.nn.CrossEntropyLoss(), optimizer=None, input_shape=(3, 32, 32), nb_classes=10)
attack = ProjectedGradientDescent(classifier, eps=0.3, eps_step=0.01, max_iter=20, targeted=False)
start_time = time.time()
for data in tqdm(trainloader):
    inputs, labels = data
    adv_inputs = attack.generate(x=inputs.numpy(), y=labels.numpy())
end_time = time.time()
print("ART PGD Attack Time:", end_time - start_time)



### CleverHans

In [ ]:
model = resnet50(pretrained=True).eval().cuda()
# CleverHans FGSM
start_time = time.time()
for data in tqdm(trainloader):
    inputs, labels = data
    inputs, labels = inputs.cuda(), labels.cuda()
    adv_inputs = projected_gradient_descent(model, inputs, eps=0.3, eps_iter=0.01, nb_iter=20, norm=np.inf, y=labels)
end_time = time.time()
print("CleverHans PGD Attack Time:", end_time - start_time)

### AdvSecureNet

In [ ]:
model = resnet50(pretrained=True).eval().cuda()
device = DeviceConfig(processor="cuda")

# Define the pgd config
pgd_config = PgdAttackConfig(
    epsilon=0.3,
    alpha=0.01,
    num_iter=20,
    targeted=False,
)

# Now we can define the attack
attack = PGD(config=pgd_config)  

# define the attacker 
attacker_config = AttackerConfig(
    model=model,
    attack= attack,
    dataloader=trainloader,
    device=device,
    return_adversarial_images=True,
)
attacker = Attacker(config=attacker_config)

start_time = time.time()
adv_imgs = attacker.execute()
end_time = time.time()
print("AdvSecureNet PGD Attack Time:", end_time - start_time)

### ARES

In [ ]:
start_time = time.time()
!python run_attack.py --gpu 0 --batchsize 128 --num_workers 2  --model_name 'resnet50' --attack_name 'pgd' --dataset 'cifar10' --data_dir ./data 
end_time = time.time()
print("Ares PGD Attack Time:", end_time - start_time)

### FoolBox

In [ ]:

attack = fb.attacks.L2PGD(steps=20)
preprocessing = dict()
bounds = (-1, 1)
fmodel = fb.PyTorchModel(model, bounds=bounds, preprocessing=preprocessing)
start_time = time.time()
for data in tqdm(trainloader):
    inputs, labels = data
    inputs, labels = inputs.cuda(), labels.cuda()
    adv_inputs = attack(fmodel, inputs, labels, epsilons=[0.3])
    
end_time = time.time()

print("Foolbox PGD Attack Time:", end_time - start_time)
    

### AdverTorch

In [ ]:
adversary = AdvertorchPGDAttack(predict=model, loss_fn=torch.nn.CrossEntropyLoss(), eps=0.3, nb_iter=20, eps_iter=0.01, clip_min=-1.0, clip_max=1.0, targeted=False)

start_time = time.time()
for data in tqdm(trainloader):
    inputs, labels = data
    inputs, labels = inputs.cuda(), labels.cuda()
    adv_inputs = adversary.perturb(inputs, labels)
end_time = time.time()

print("Advertorch PGD Attack Time:", end_time - start_time)